# Design



### Challenge Name: fw_protect (/embsec/design/fw_protect)

The purpose of this tool will be to secure the firmware. Eventually,
you will want to keep the firmware confidential and be able to verify its integrity and authenticity. But for now, 
you will implement a firmware protect tool that does not add security. This tool should simply:

    1. Pack the version "0" into a little-endian short
    2. Load the firmware binary from firmware.bin
    3. Pack the size of the firmware into a little-endian short
    4. Append the release message "First version of firmware." to end of firmware (make sure it is null-terminated)
    5. Build and send a binary blob in the following format to the grader:
    
        [ 0x2 ]      [ 0x2 ]      [ variable ]           [ variable ]
        -------------------------------------------------------------------------
        version | firmware size |   firmware   |   release message + null-byte   




In [11]:
from embsec import Serial
import struct, os
#sent from firmware to bootloader





def fw_protect():
    ser = Serial("/embsec/design/fw_protect")
    # Your code goes here!
    with open("firmware_0.bin", 'rb') as f: #rb = read as bytes
        size = os.stat("firmware_0.bin").st_size
        ser.write(struct.pack(f"<HH{size}s27s", 0, size, f.read(), "First version of firmware.\0".encode("utf-8"))) 
        #f allows you to put variables in brakets (the {size})
        #< --> little endian 
        #HH --> unsigned short
        #s --> type specifier (string)
        #27s --> says how long the string is
        return ser.read_until() #reads the serial input (from the bus) until a new line character

fw_protect()


b'embsec{fw_protect_b9d4326240c079e8}\n'

### Challenge Name: fw_update (/embsec/design/fw_update)

For this lesson you will implement a basic update tool for the design challenge. The tool will be used to update the device firmware. You should focus on reading in the firmware blob file, and constructing a set of frames to send to the bootloader. Pay attention to the structure of these frames, as the bootloader is expecting a certain size and format.

The bootloader is expecting frames of the following format:

      [ 0x2 ]    [ variable ]
----------------------------
     data size |   data... 

Process:
1. Read in binary blob
2. Send the file metadata over serial. Remeber from fw_protect that the metadata is the first 4 bytes of the file!
3. Send a 'U' for update mode & receive a confirmation 'U'
4. Construct frames from blob, note that the size is the size of the data field!
5. Send frames sequentially to boot loader. You need to wait for a zero byte between frames! This is an OK heartbeat response.
6. To finish, you need to send a frame with the size field set to zero. Feel free to leave the data field empty.
7. Read for a flag!

One last thing! Frames are packed big-endian. (



In [5]:
from embsec import Serial
import struct, os

def fw_update():
    ser = Serial("/embsec/design/fw_update")
    # Your code goes here!
    
    with open("firmware_1.blob", 'rb') as f: #defines file object
    
        ser.write('U'.encode('UTF-8')) #sends u
        if(ser.read() == 'U'.encode('UTF-8')): #gets u
            ser.write(f.read(4)) #sends metadata
            size = os.stat("firmware_1.blob").st_size - 4
            
        
            k = 1
            while size > 14:
               # print(f"loop runthrough #{k}")
                packet = struct.pack(">H14s", 14, f.read(14))
                ser.write(packet)
            #    print(f"loop runthrough #{k} midway")
                size -= 14
            #    print(ser.read())
               # print(f"loop runthrough #{k} end")
                k+=1
            if size>0:
                ser.write(struct.pack(f">H{size}s", size, f.read()))
             #   print(ser.read())
            ser.write(struct.pack(">H0b", 0))
      #  print("here")
        return ser.read_until()
    

    

fw_update()


b'\x00\x00\x00\x00\x00\x00embsec{fw_update_20ef48882974720a}\n'